In [8]:
# %% imports

import os
import json
import subprocess
import pickle
import sys

from sklearn.utils.estimator_checks import check_estimator
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.feature_selection import mutual_info_classif
import ReadData as ReadData
import cString2json as cString2json
import json2graphNoLeafEdgesWithSplitValues as json2graphNoLeafEdgesWithSplitValues
from fitModels import fitModels
import DecisionSnippetFeatures as DecisionSnippetFeatures
import pruning
import Forest
import datetime
from util import writeToReport
import numpy as np
# %% Parameters. 

dataPath = "../data/"
forestsPath = "../tmp/forests/"
snippetsPath = "../tmp/snippets/"
resultsPath = "../tmp/results/"
reportsPath = "../tmp/reports/"

# current valid options are ['sensorless', 'satlog', 'mnist', 'magic', 'spambase', 'letter', 'bank', 'adult', 'drinking']

#dataset = sys.argv[1]
dataset = 'adult'
print(dataset)
#dataSet = 'satlog'
# dataSet = 'adult'
# dataSet = 'drinking'

# possible forest_types ['RF', 'DT', 'ET']
forest_types = ['RF']
forest_depths = [5]
sigma_values = [0.3]
forest_sizes = [25]
maxPatternSize = 1
#minThreshold = int(sys.argv[2])
#maxThreshold = int(sys.argv[3])

minThreshold = 1
maxThreshold = 1

#edge_thresholds = [1.0,0.975,0.95,0.925,0.9,0.85,0.8,0.7,0.6]
edge_thresholds = [1.0]

#mode = int(sys.argv[2])
mode = 2
if (mode == 1):
    
    run_fit_models = True
    run_mining = True
    run_training = False
    run_eval = False
    
if (mode == 2):
    
    run_fit_models = False
    run_mining = False
    run_training = True
    run_eval = False
    
#patterns_thresholds = [1,2,3]

scoring_function = 'accuracy'

# learners that are to be used on top of Decision Snippet Features
learners = {'NB': MultinomialNB,
            'SVM': LinearSVC,
            'LR': LogisticRegression}

#learners = {'NB': MultinomialNB}

# specify parameters that are given at initialization
learners_parameters = {'NB': {},
                       'SVM': {'max_iter': 10000},
                       'LR': {'max_iter': 10000}}

#learners_parameters = {'NB': {}}


# for quick debugging, let the whole thing run once. Afterwards, you may deactivate individual steps
# each step stores its output for the subsequent step(s) to process

verbose = True

fitting_models_time = datetime.timedelta()
pruning_time = datetime.timedelta()



# %% load data

X_train, Y_train = ReadData.readData(dataset, 'train', dataPath)
X_test, Y_test = ReadData.readData(dataset, 'test', dataPath)
X = X_train

arr = mutual_info_classif(X_train,Y_train)
print(arr)


adult
[9.38522692e-02 0.00000000e+00 3.83311415e-03 1.05216119e-02
 1.02228721e-02 7.23444040e-03 2.59079021e-03 0.00000000e+00
 4.95209759e-03 3.59536539e-02 2.52651830e-02 1.51311745e-03
 9.91172783e-03 9.87822748e-03 1.25643347e-02 0.00000000e+00
 5.09179469e-04 7.49263753e-03 0.00000000e+00 1.11986371e-03
 1.41327857e-02 4.24001879e-03 1.12664787e-04 6.97334627e-03
 0.00000000e+00 7.49367155e-04 7.80903469e-02 1.44384741e-01
 1.33040061e-02 9.26007850e-02 6.73173941e-03 3.64499305e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 6.22195332e-03
 2.86125004e-02 4.34751307e-03 2.75823440e-02 1.72109759e-02
 1.14701808e-02 6.57170305e-03 8.05401830e-03 6.11903282e-03
 5.05694139e-03 7.80242472e-03 0.00000000e+00 8.83028306e-03
 6.64493825e-03 5.60260772e-02 1.04148488e-01 2.42904312e-02
 4.63969769e-03 1.94982389e-02 8.76054264e-03 0.00000000e+00
 1.81324675e-03 0.00000000e+00 1.01878419e-02 4.01649081e-02
 3.36525090e-02 8.65535565e-02 3.64912224e-02 5.82834595e-02]


In [9]:
from collections import defaultdict

    
'''
for feature in range(len(X_train[0])):
    s = set()
    for x in X_train:
        s.add(x[feature])
'''
Y_train_1 = sum(Y_train)
Y_train_0 = len(Y_train) - Y_train_1

index = 0        
    
for feature in range(len(X_train[0])):
    #print('Feature: ' + str(feature))
    
    values = set(X_train[:,feature])
    for value in values:
        #print('Value: ' + str(value))
        #print(values)
        class_counts = defaultdict(int)
        for sample, y in zip(X_train, Y_train):
            if(sample[feature] <= value):
                class_counts[y] += 1
        #print(class_counts) 
        if (class_counts[0] <= class_counts[1]):
            index = 1
        else:
            index = 0
            
        majority = class_counts[index]
        minority = class_counts[1-index]
            
        if (index == 1):
            sub_error = Y_train_1 - majority
        else:
            sub_error = Y_train_0 - majority
        errors_count = minority + sub_error 
        #print(errors_count)
        if (errors_count < 5500):
            print('Feature: ' + str(feature) + ', Value: ' + str(value) + ', Errors count: ' + str(errors_count))
                
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

Feature: 26, Value: 10, Errors count: 5316
Feature: 26, Value: 11, Errors count: 5345
Feature: 26, Value: 12, Errors count: 5349
Feature: 27, Value: 0, Errors count: 3958
Feature: 29, Value: 0, Errors count: 5385
Feature: 50, Value: 0, Errors count: 4383
